In [1]:
import os
import importlib
from utility import Pi_util

In [2]:
test_path: str = "/Volumes/EXTERNAL1/withTimings_5hz_0NDF"

importlib.reload(Pi_util)
Pi_util.parse_chunk_paths(test_path)

[{'MS': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst0_MS_readings'],
  'Pupil': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst0_pupil'],
  'World': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst0_world',
   '/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst0_world_settingsHistory.csv'],
  'Sunglasses': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst0_sunglasses.csv']},
 {'MS': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst1_MS_readings'],
  'Pupil': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst1_pupil'],
  'World': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst1_world',
   '/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst1_world_settingsHistory.csv'],
  'Sunglasses': ['/Volumes/EXTERNAL1/withTimings_5hz_0NDF/withTimings_5hz_0NDF_burst1_sunglasses.csv']},
 {'MS': ['/Volumes/EXTERNAL1/withTimings_5hz